# Creating a Sentiment Analysis Web App
## Using PyTorch and SageMaker

_Deep Learning Nanodegree Program | Deployment_

---

Now that we have a basic understanding of how SageMaker works we will try to use it to construct a complete project from end to end. Our goal will be to have a simple web page which a user can use to enter a movie review. The web page will then send the review off to our deployed model which will predict the sentiment of the entered review.

## Instructions

Some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this notebook. You will not need to modify the included code beyond what is requested. Sections that begin with '**TODO**' in the header indicate that you need to complete or implement some portion within them. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `# TODO: ...` comment. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions for you to answer which relate to the task and your implementation. Each section where you will answer a question is preceded by a '**Question:**' header. Carefully read each question and provide your answer below the '**Answer:**' header by editing the Markdown cell.

> **Note**: Code and Markdown cells can be executed using the **Shift+Enter** keyboard shortcut. In addition, a cell can be edited by typically clicking it (double-click for Markdown cells) or by pressing **Enter** while it is highlighted.

## General Outline

Recall the general outline for SageMaker projects using a notebook instance.

1. Download or otherwise retrieve the data.
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.

For this project, you will be following the steps in the general outline with some modifications. 

First, you will not be testing the model in its own step. You will still be testing the model, however, you will do it by deploying your model and then using the deployed model by sending the test data to it. One of the reasons for doing this is so that you can make sure that your deployed model is working correctly before moving forward.

In addition, you will deploy and use your trained model a second time. In the second iteration you will customize the way that your trained model is deployed by including some of your own code. In addition, your newly deployed model will be used in the sentiment analysis web app.

## Step 1: Downloading the data

As in the XGBoost in SageMaker notebook, we will be using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [78]:
#%mkdir ../data
#!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2019-01-20 22:35:18--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  10.2MB/s    in 9.9s    

2019-01-20 22:35:28 (8.11 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Step 2: Preparing and Processing the data

Also, as in the XGBoost notebook, we will be doing some initial data processing. The first few steps are the same as in the XGBoost example. To begin with, we will read in each of the reviews and combine them into a single input structure. Then, we will split the dataset into a training set and a testing set.

In [1]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [2]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.

In [39]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [40]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Now that we have our training and testing sets unified and prepared, we should do a quick check and see an example of the data our model will be trained on. This is generally a good idea as it allows you to see how each of the further processing steps affects the reviews and it also ensures that the data has been loaded correctly.

In [6]:
print(train_X[100])
print(train_y[100])

Surviving Christmas is a surprisingly funny movie especially considering the bad publicity when it was first released. Ben Affleck is funny as an obnoxious millionaire who pays the family that occupies his childhood home to be his family for Christmas. He then drives the family crazy with overindulgence for Christmas cheer. I have not been a Ben Affleck fan in the past (though I did like Daredevil and Paycheck) but here he is well cast in this role. I also like Christina Applegate as the daughter in the family who can't stand Affleck's character at first. Sure you can see where this movie is going but you don't care. Ignore what the critics say and rent this movie out because it is funnier than a lot of Christmas movies.
1


The first step in processing the reviews is to make sure that any html tags that appear should be removed. In addition we wish to tokenize our input, that way words such as *entertained* and *entertaining* are considered the same with regard to sentiment analysis.

In [291]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

The `review_to_words` method defined above uses `BeautifulSoup` to remove any html tags that appear and uses the `nltk` package to tokenize the reviews. As a check to ensure we know how everything is working, try applying `review_to_words` to one of the reviews in the training set.

In [42]:
# TODO: Apply review_to_words to a review (train_X[100] or any other review)
#print(review_to_words(train_X[100]))
a = review_to_words(train_X[100])
a.sort()
print(train_X[100])
print(a)

Ok, so it may not be the award-winning "movie of the year" type-film (apart from the brilliant soundtrack that I think won a few awards), but it is a really great film about 'The Kid' (Prince / O( take your pick) and the happenings around him living in Minneapolis, playing his music. The music is absolutely superb, in my opinion you HAVE to own this soundtrack, it is truly a classic and sums up the eighties sounds and feel in a wonderful fashion. And the movie itself plays out a nice plot, it's worth seeing over and over again, espeically if you like Prince / O (which I do) of course.
['absolut', 'apart', 'around', 'award', 'award', 'brilliant', 'classic', 'cours', 'eighti', 'espeic', 'fashion', 'feel', 'film', 'film', 'great', 'happen', 'kid', 'like', 'live', 'may', 'minneapoli', 'movi', 'movi', 'music', 'music', 'nice', 'ok', 'opinion', 'pick', 'play', 'play', 'plot', 'princ', 'princ', 'realli', 'see', 'sound', 'soundtrack', 'soundtrack', 'sum', 'superb', 'take', 'think', 'truli', 't

In [43]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'
a = review_to_words(test_review)
a.sort()
print(a)

['adventur', 'basic', 'best', 'charm', 'combin', 'fair', 'film', 'life', 'love', 'movi', 'one', 'pleasur', 'rather', 'simplest', 'storylin', 'transcend', 'unmitig', 'usual', 'weekend', 'wit']


**Question:** Above we mentioned that `review_to_words` method removes html formatting and allows us to tokenize the words found in a review, for example, converting *entertained* and *entertaining* into *entertain* so that they are treated as though they are the same word. What else, if anything, does this method do to the input?

**Answer:**
In addtion to tokenizing and removing httml tags, It also reduces words to their stem word.

The method below applies the `review_to_words` method to each of the reviews in the training and testing datasets. In addition it caches the results. This is because performing this processing step can take a long time. This way if you are unable to complete the notebook in the current session, you can come back without needing to process the data a second time.

In [44]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [71]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


## Transform the data

In the XGBoost notebook we transformed the data from its word representation to a bag-of-words feature representation. For the model we are going to construct in this notebook we will construct a feature representation which is very similar. To start, we will represent each word as an integer. Of course, some of the words that appear in the reviews occur very infrequently and so likely don't contain much information for the purposes of sentiment analysis. The way we will deal with this problem is that we will fix the size of our working vocabulary and we will only include the words that appear most frequently. We will then combine all of the infrequent words into a single category and, in our case, we will label it as `1`.

Since we will be using a recurrent neural network, it will be convenient if the length of each review is the same. To do this, we will fix a size for our reviews and then pad short reviews with the category 'no word' (which we will label `0`) and truncate long reviews.

### (TODO) Create a word dictionary

To begin with, we need to construct a way to map words that appear in the reviews to integers. Here we fix the size of our vocabulary (including the 'no word' and 'infrequent' categories) to be `5000` but you may wish to change this to see how it affects the model.

> **TODO:** Complete the implementation for the `build_dict()` method below. Note that even though the vocab_size is set to `5000`, we only want to construct a mapping for the most frequently appearing `4998` words. This is because we want to reserve the special labels `0` for 'no word' and `1` for 'infrequent word'.

In [72]:
import numpy as np
from collections import Counter
import sys

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    
    word_count = Counter() # A dict storing the words that appear in the reviews along with how often they occur
    for sentence in data:
        words = sentence[0].split(" ")
        for word in words:
            word_count[word] += 1
    
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    freq_ordered_words = word_count.most_common();
    sorted_words = list(map(lambda x: x[0], freq_ordered_words))
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [73]:
word_dict = build_dict(train_X)

**Question:** What are the five most frequently appearing (tokenized) words in the training set? Does it makes sense that these words appear frequently in the training set?

**Answer:**

In [265]:
# TODO: Use this space to determine the five most frequently appearing words in the training set.
list(word_dict.keys())[:5]

['movi', 'film', 'one', 'first', 'saw']

### Save `word_dict`

Later on when we construct an endpoint which processes a submitted review we will need to make use of the `word_dict` which we have created. As such, we will save it to a file now for future use.

In [75]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [76]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the reviews

Now that we have our word dictionary which allows us to transform the words appearing in the reviews into integers, it is time to make use of it and convert our reviews to their integer sequence representation, making sure to pad or truncate to a fixed length, which in our case is `500`.

In [303]:
import sys

#method takes word_dictionary-> ordered_position e,g 2,3.., sentence in list, pads with 0s after pad max
#1 is given to infrequent words not in word_dictionary
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
       # print("word_index : {}, word {}".format(word_index, word))
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
   # print(working_sentence, min(len(sentence), pad))
    #sys.exit(0)
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
    return np.array(result), np.array(lengths)

In [78]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

In [266]:
print(train_X[:5],train_X_len[:5])

[[  35  108 1688 ...    0    0    0]
 [1963  525    1 ...    0    0    0]
 [ 296    1   34 ...    0    0    0]
 [ 172 1119    8 ...    0    0    0]
 [  18  203  462 ...    0    0    0]] [56 72 19 70 74]


In [174]:

test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'
a = review_to_words(test_review)
sample_X, sample_X_len =convert_and_pad(word_dict, a, pad=500)
print(sample_X)

[1, 420, 172, 61, 3, 4, 1373, 285, 161, 598, 9, 2606, 2, 1, 87, 1279, 831, 1357, 1, 477, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

As a quick check to make sure that things are working as intended, check to see what one of the reviews in the training set looks like after having been processeed. Does this look reasonable? What is the length of a review in the training set?

In [175]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
print(len(train_X), len(train_X_len))
print(train_X[:5], train_X_len[:5])

25000 25000
[[  35  108 1688 ...    0    0    0]
 [1963  525    1 ...    0    0    0]
 [ 296    1   34 ...    0    0    0]
 [ 172 1119    8 ...    0    0    0]
 [  18  203  462 ...    0    0    0]] [56 72 19 70 74]


**Question:** In the cells above we use the `preprocess_data` and `convert_and_pad_data` methods to process both the training and testing set. Why or why not might this be a problem?

**Answer:**
These methods transform the input data by maximizing the information by reducing the complexity, noise and length of sentences. Each review starts as a string and is transformed into a reduced list of common stem words. It might be a problem that we are fitlering out some common stop words if they contribute to the setence context in which sentiment is determined, and RNNs (LSTM) learn context in sequences.

The word dictionary is based on the training data, and applied to both the test and training data. Of course we would expect a better result when applied to the training set for which it is made. But applying the training-based-dictionary is good for testing how representative the training data is in production.

The way the code is written the transform can be re-applied to the same variable i.e X <= F[X], this can cause bugs if the code is executed twice.

The functions convert_and_pad, review_to_words applied to training and test data sets will also need to be applied to user input values on a deployed server which may be challenging. E.G review_to_words will need to download the "stopwords". 


## Step 3: Upload the data to S3

As in the XGBoost notebook, we will need to upload the training dataset to S3 in order for our training code to access it. For now we will save it locally and we will upload to S3 later on.

### Save the processed training dataset locally

It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In our case, each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of `500` integers representing the words in the review.

In [82]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data


Next, we need to upload the training data to the SageMaker default S3 bucket so that we can provide access to it while training our model.

In [83]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [84]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

**NOTE:** The cell above uploads the entire contents of our data directory. This includes the `word_dict.pkl` file. This is fortunate as we will need this later on when we create an endpoint that accepts an arbitrary review. For now, we will just take note of the fact that it resides in the data directory (and so also in the S3 training bucket) and that we will need to make sure it gets saved in the model directory.

## Step 4: Build and Train the PyTorch Model

In the XGBoost notebook we discussed what a model is in the SageMaker framework. In particular, a model comprises three objects

 - Model Artifacts,
 - Training Code, and
 - Inference Code,
 
each of which interact with one another. In the XGBoost example we used training and inference code that was provided by Amazon. Here we will still be using containers provided by Amazon with the added benefit of being able to include our own custom code.

We will start by implementing our own neural network in PyTorch along with a training script. For the purposes of this project we have provided the necessary model object in the `model.py` file, inside of the `train` folder. You can see the provided implementation by running the cell below.

In [85]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

The important takeaway from the implementation provided is that there are three parameters that we may wish to tweak to improve the performance of our model. These are the embedding dimension, the hidden dimension and the size of the vocabulary. We will likely want to make these parameters configurable in the training script so that if we wish to modify them we do not need to modify the script itself. We will see how to do this later on. To start we will write some of the training code in the notebook so that we can more easily diagnose any issues that arise.

First we will load a small portion of the training data set to use as a sample. It would be very time consuming to try and train the model completely in the notebook as we do not have access to a gpu and the compute instance that we are using is not particularly powerful. However, we can work on a small bit of the data to get a feel for how our training script is behaving.

In [86]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

### (TODO) Writing the training method

Next we need to write the training code itself. This should be very similar to training methods that you have written before to train PyTorch models. We will leave any difficult aspects such as model saving / loading and parameter loading until a little later.

In [87]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        show_every_n_batches = 40
        total_loss = 0
        batch_i = 0
        for batch in train_loader:         
            batch_X, batch_y = batch     
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            out = model.forward(batch_X)
            loss = loss_fn(out, batch_y)
            n_batches = len(train_loader.dataset)//batch_X.shape[0]
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            total_loss += loss.data.item()
            if batch_i % show_every_n_batches == 0:
                print('Batch : {}/{} Epoch: {}/{}  Loss: {:6.4f}, Av Loss: {:6.4f}\n'.format(batch_i, n_batches, epoch, epochs+1, total_loss, total_loss/(batch_i+1)))
                #print('Batch : {}/{} Epoch: {:>4}/{:<4}  Loss: {:6.4f}, Av Loss: {:6.4f}\n'.format(batch_i, n_batches, epoch, epochs+1, total_loss, total_loss/(batch_i+1)))
            total_loss += loss.data.item()
            batch_i += 1
        print("Epoch: {}, BCELoss: {:6.2f}".format(epoch, total_loss / len(train_loader)))

Supposing we have the training method above, we will test that it is working by writing a bit of code in the notebook that executes our training method on the small sample training set that we loaded earlier. The reason for doing this in the notebook is so that we have an opportunity to fix any errors that arise early when they are easier to diagnose.

In [88]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()
print(model)
train(model, train_sample_dl, 5, optimizer, loss_fn, device)

LSTMClassifier(
  (embedding): Embedding(5000, 32, padding_idx=0)
  (lstm): LSTM(32, 100)
  (dense): Linear(in_features=100, out_features=1, bias=True)
  (sig): Sigmoid()
)
Batch : 0/5 Epoch: 1/6  Loss: 0.6870, Av Loss: 0.6870

Epoch: 1, BCELoss:   1.39
Batch : 0/5 Epoch: 2/6  Loss: 0.6750, Av Loss: 0.6750

Epoch: 2, BCELoss:   1.37
Batch : 0/5 Epoch: 3/6  Loss: 0.6669, Av Loss: 0.6669

Epoch: 3, BCELoss:   1.35
Batch : 0/5 Epoch: 4/6  Loss: 0.6582, Av Loss: 0.6582

Epoch: 4, BCELoss:   1.34
Batch : 0/5 Epoch: 5/6  Loss: 0.6478, Av Loss: 0.6478

Epoch: 5, BCELoss:   1.31


In order to construct a PyTorch model using SageMaker we must provide SageMaker with a training script. We may optionally include a directory which will be copied to the container and from which our training code will be run. When the training container is executed it will check the uploaded directory (if there is one) for a `requirements.txt` file and install any required Python libraries, after which the training script will be run.

### (TODO) Training the model

When a PyTorch model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained. Inside of the `train` directory is a file called `train.py` which has been provided and which contains most of the necessary code to train our model. The only thing that is missing is the implementation of the `train()` method which you wrote earlier in this notebook.

**TODO**: Copy the `train()` method written above and paste it into the `train/train.py` file where required.

The way that SageMaker passes hyperparameters to the training script is by way of arguments. These arguments can then be parsed and used in the training script. To see how this is done take a look at the provided `train/train.py` file.

In [89]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.m5.xlarge',
                    hyperparameters={
                        'epochs': 40,
                        'hidden_dim': 200,
                    })

#train_instance_type='ml.p2.xlarge',

In [90]:
estimator.fit({'training': input_data})

INFO:sagemaker:Creating training-job with name: sagemaker-pytorch-2019-01-24-09-20-49-390


2019-01-24 09:20:50 Starting - Starting the training job...
2019-01-24 09:20:51 Starting - Launching requested ML instances......
2019-01-24 09:21:54 Starting - Preparing the instances for training...
2019-01-24 09:22:40 Downloading - Downloading input data..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-01-24 09:22:59,808 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-01-24 09:22:59,811 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-01-24 09:22:59,823 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-01-24 09:23:02,832 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-01-24 09:23:03,051 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-01-24 09:23:03,051 sagemaker-containers INFO     Generating setup.cfg
2019-01-24 0

Batch : 0/48 Epoch: 1/41  Loss: 0.6949, Av Loss: 0.6949

Batch : 10/48 Epoch: 1/41  Loss: 14.4856, Av Loss: 1.3169

Batch : 20/48 Epoch: 1/41  Loss: 28.0616, Av Loss: 1.3363

Batch : 30/48 Epoch: 1/41  Loss: 41.0142, Av Loss: 1.3230

Batch : 40/48 Epoch: 1/41  Loss: 53.5673, Av Loss: 1.3065

Epoch: 1, BCELoss:   1.31
Batch : 0/48 Epoch: 2/41  Loss: 0.6246, Av Loss: 0.6246

Batch : 10/48 Epoch: 2/41  Loss: 12.5352, Av Loss: 1.1396

Batch : 20/48 Epoch: 2/41  Loss: 24.1352, Av Loss: 1.1493

Batch : 30/48 Epoch: 2/41  Loss: 35.2217, Av Loss: 1.1362

Batch : 40/48 Epoch: 2/41  Loss: 46.0873, Av Loss: 1.1241

Epoch: 2, BCELoss:   1.13
Batch : 0/48 Epoch: 3/41  Loss: 0.5797, Av Loss: 0.5797

Batch : 10/48 Epoch: 3/41  Loss: 11.4692, Av Loss: 1.0427

Batch : 20/48 Epoch: 3/41  Loss: 21.9177, Av Loss: 1.0437

Batch : 30/48 Epoch: 3/41  Loss: 31.5400, Av Loss: 1.0174

Batch : 40/48 Epoch: 3/41  Loss: 42.2271, Av Loss: 1.0299

Epoch: 3, BCELoss:   1.04
Batch : 0/48 Epoch: 4/41  Loss: 0.4989, Av 

Epoch: 22, BCELoss:   0.44
Batch : 0/48 Epoch: 23/41  Loss: 0.2100, Av Loss: 0.2100

Batch : 10/48 Epoch: 23/41  Loss: 4.6322, Av Loss: 0.4211

Batch : 20/48 Epoch: 23/41  Loss: 9.3840, Av Loss: 0.4469

Batch : 30/48 Epoch: 23/41  Loss: 14.9041, Av Loss: 0.4808

Batch : 40/48 Epoch: 23/41  Loss: 19.9407, Av Loss: 0.4864

Epoch: 23, BCELoss:   0.50
Batch : 0/48 Epoch: 24/41  Loss: 0.2666, Av Loss: 0.2666

Batch : 10/48 Epoch: 24/41  Loss: 5.1862, Av Loss: 0.4715

Batch : 20/48 Epoch: 24/41  Loss: 9.9405, Av Loss: 0.4734

Batch : 30/48 Epoch: 24/41  Loss: 14.6838, Av Loss: 0.4737

Batch : 40/48 Epoch: 24/41  Loss: 19.1350, Av Loss: 0.4667

Epoch: 24, BCELoss:   0.48
Batch : 0/48 Epoch: 25/41  Loss: 0.2778, Av Loss: 0.2778

Batch : 10/48 Epoch: 25/41  Loss: 5.2962, Av Loss: 0.4815

Batch : 20/48 Epoch: 25/41  Loss: 9.5797, Av Loss: 0.4562

Batch : 30/48 Epoch: 25/41  Loss: 13.7194, Av Loss: 0.4426

Batch : 40/48 Epoch: 25/41  Loss: 17.5235, Av Loss: 0.4274

Epoch: 25, BCELoss:   0.44
Batc

## Step 5: Testing the model

As mentioned at the top of this notebook, we will be testing this model by first deploying it and then sending the testing data to the deployed endpoint. We will do this so that we can make sure that the deployed model is working correctly.

## Step 6: Deploy the model for testing

Now that we have trained our model, we would like to test it to see how it performs. Currently our model takes input of the form `review_length, review[500]` where `review[500]` is a sequence of `500` integers which describe the words present in the review, encoded using `word_dict`. Fortunately for us, SageMaker provides built-in inference code for models with simple inputs such as this.

There is one thing that we need to provide, however, and that is a function which loads the saved model. This function must be called `model_fn()` and takes as its only parameter a path to the directory where the model artifacts are stored. This function must also be present in the python file which we specified as the entry point. In our case the model loading function has been provided and so no changes need to be made.

**NOTE**: When the built-in inference code is run it must import the `model_fn()` method from the `train.py` file. This is why the training code is wrapped in a main guard ( ie, `if __name__ == '__main__':` )

Since we don't need to change anything in the code that was uploaded during training, we can simply deploy the current model as-is.

**NOTE:** When deploying a model you are asking SageMaker to launch an compute instance that will wait for data to be sent to it. As a result, this compute instance will continue to run until *you* shut it down. This is important to know since the cost of a deployed endpoint depends on how long it has been running for.

In other words **If you are no longer using a deployed endpoint, shut it down!**

**TODO:** Deploy the trained model.

In [308]:
# TODO: Deploy the trained model
# TODO: Create a transformer object from the trained model. Using an instance count of 1 and an instance type of ml.m4.xlarge
#       should be more than enough.
model_transformer = estimator.transformer(instance_count = 1, instance_type = 'ml.m5.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-pytorch-2019-01-24-09-20-49-390


In [309]:
predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m5.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-pytorch-2019-01-24-09-20-49-390
INFO:sagemaker:Creating endpoint with name sagemaker-pytorch-2019-01-24-09-20-49-390


---------------------------------------------------------------------------!

## Step 7 - Use the model for testing

Once deployed, we can read in the test data and send it off to our deployed model to get some results. Once we collect all of the results we can determine how accurate our model is.

In [313]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [314]:
# We split the data into chunks and send each chunk seperately, accumulating the results.
import sys
def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        #print(type(array))
       # print(array.shape)
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [315]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.8408

**Question:** How does this model compare to the XGBoost model you created earlier? Why might these two models perform differently on this dataset? Which do *you* think is better for sentiment analysis?

**Answer:**
XGGBoost performed marginally better at 0.85508 as compared to the RNNs 0.8408 on test data sets, both models may be converging toward optimal/saturated accuracy levels. 
I believe both models are likely struggling on the 0.15 - which I hypothesize consists of more complex reviews with neither strongly positive or negative reviews.

The models have different representations of the input data.  XGBoost uses Bag Of Words, mathematically this is a set of words with count data. It explicitly uses frequency information as it sends a distribution of word count data into the model, and becomes a statistical classifier.  RNN uses an ordered sequence of words for input. Unlike XGBoost it does not explicity encode frequency as each index represents a category. Instead the frequency is governed implicity by the number of times the word appears in the sequence. But because it applies an LSTM to the sequence it may learn context from the relative orderings of words.

As XGBoost uses explicity frequency information only, while RNN uses both context and implicit frequency, I would expect both models to perform well on a large proportion of strong positive/negative reviews E.G 0.85, but RNN to perform better on more complex reviews within 0.15. To test this hypothesis, I would need to train larger RNN models for a longer period. While I believe XGBoost will saturate to maximum accuracy level despite using greater tree depth.


### (TODO) More testing

We now have a trained model which has been deployed and which we can send processed reviews to and which returns the predicted sentiment. However, ultimately we would like to be able to send our model an unprocessed review. That is, we would like to send the review itself as a string. For example, suppose we wish to send the following review to our model.

In [317]:
test_review ='The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

In [321]:
test_review1 = 'This movie is not your ordinary Hollywood flick. It has a great and deep message. This movie has a foundation and just kept on being built on from their and that foundation is hope. Other than just the message of this movie the acting was phenomenal. Tim Robbins gave one of the greatest performances ever. He was inspiring, intelligent and most of all positive. His performance just made me smile. Robbins plays Andy Dufresne who was wrongfully convicted of murdering his wife and her lover. He is gets to life sentences but yet never gives up hope. In he becomes friends with Ellis Boyd "Red" Redding played by Morgan Freeman. Freeman who gives the finest performance of his career has unlike Robbins lost hope. He is in deep regret of the crime that he committed. His way of deflecting the pain away is by trying to not feel anything at all. With his friendship with Andy he learns that without our hopes and dreams we have nothing. Andy also becomes friends with the rest of Red\'s group. James Whitmore also gave a great performance as Brooks Halten who gets out of prison parole but in the words of Red he has been \'institutionalized\'. The directing by Frank Darabont was just magnificent. He kept this movie at a great steady pace along with the writing and great cinematography. He portrayed prison life in such a horrifying way, but not in terms of the physical pain but the stress and pain that wares mentally on the inmates, some of which deserve a second chance. Whatever you do, don\'t listen to the people who say this movie is overrated because this is one of the most inspiring and greatest movies ever. It has everything you could possibly want.'

The question we now need to answer is, how do we send this review to our model?

Recall in the first section of this notebook we did a bunch of data processing to the IMDb dataset. In particular, we did two specific things to the provided reviews.
 - Removed any html tags and stemmed the input
 - Encoded the review as a sequence of integers using `word_dict`
 
In order process the review we will need to repeat these two steps.

**TODO**: Using the `review_to_words` and `convert_and_pad` methods from section one, convert `test_review` into a numpy array `test_data` suitable to send to our model. Remember that our model expects input of the form `review_length, review[500]`.

In [318]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data
import numpy
#test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'
a = review_to_words(test_review)

sample_X, sample_X_len =convert_and_pad(word_dict, a, pad=500)
sample_X=numpy.asarray(sample_X)
test_data = numpy.reshape(sample_X, (1,-1))


Now that we have processed the review, we can send the resulting array to our model to predict the sentiment of the review.

In [319]:
predictor.predict(test_data)

array(0.60073024, dtype=float32)

In [322]:
a = review_to_words(test_review1)
#false negative
sample_X, sample_X_len =convert_and_pad(word_dict, a, pad=500)
sample_X=numpy.asarray(sample_X)
test_data = numpy.reshape(sample_X, (1,-1))
predictor.predict(test_data)

array(0.32397455, dtype=float32)

Since the return value of our model is close to `1`, we can be certain that the review we submitted is positive.

### Delete the endpoint

Of course, just like in the XGBoost notebook, once we've deployed an endpoint it continues to run until we tell it to shut down. Since we are done using our endpoint for now, we can delete it.

In [323]:
estimator.delete_endpoint()

INFO:sagemaker:Deleting endpoint with name: sagemaker-pytorch-2019-01-24-09-20-49-390


## Step 6 (again) - Deploy the model for the web app

Now that we know that our model is working, it's time to create some custom inference code so that we can send the model a review which has not been processed and have it determine the sentiment of the review.

As we saw above, by default the estimator which we created, when deployed, will use the entry script and directory which we provided when creating the model. However, since we now wish to accept a string as input and our model expects a processed review, we need to write some custom inference code.

We will store the code that we write in the `serve` directory. Provided in this directory is the `model.py` file that we used to construct our model, a `utils.py` file which contains the `review_to_words` and `convert_and_pad` pre-processing functions which we used during the initial data processing, and `predict.py`, the file which will contain our custom inference code. Note also that `requirements.txt` is present which will tell SageMaker what Python libraries are required by our custom inference code.

When deploying a PyTorch model in SageMaker, you are expected to provide four functions which the SageMaker inference container will use.
 - `model_fn`: This function is the same function that we used in the training script and it tells SageMaker how to load our model.
 - `input_fn`: This function receives the raw serialized input that has been sent to the model's endpoint and its job is to de-serialize and make the input available for the inference code.
 - `output_fn`: This function takes the output of the inference code and its job is to serialize this output and return it to the caller of the model's endpoint.
 - `predict_fn`: The heart of the inference script, this is where the actual prediction is done and is the function which you will need to complete.

For the simple website that we are constructing during this project, the `input_fn` and `output_fn` methods are relatively straightforward. We only require being able to accept a string as input and we expect to return a single value as output. You might imagine though that in a more complex application the input or output may be image data or some other binary data which would require some effort to serialize.

### (TODO) Writing inference code

Before writing our custom inference code, we will begin by taking a look at the code which has been provided.

In [252]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import re
from bs4 import BeautifulSoup

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_in

As mentioned earlier, the `model_fn` method is the same as the one provided in the training code and the `input_fn` and `output_fn` methods are very simple and your task will be to complete the `predict_fn` method. Make sure that you save the completed file as `predict.py` in the `serve` directory.

**TODO**: Complete the `predict_fn()` method in the `serve/predict.py` file.

### Deploying the model

Now that the custom inference code has been written, we will create and deploy our model. To begin with, we need to construct a new PyTorchModel object which points to the model artifacts created during training and also points to the inference code that we wish to use. Then we can call the deploy method to launch the deployment container.

**NOTE**: The default behaviour for a deployed PyTorch model is to assume that any input passed to the predictor is a `numpy` array. In our case we want to send a string so we need to construct a simple wrapper around the `RealTimePredictor` class to accomodate simple strings. In a more complicated situation you may want to provide a serialization object, for example if you wanted to sent image data.

In [236]:
import numpy as np
l = 1
x = np.array([1, 2, 3,4,5,6,7,8,9,10], dtype=np.float)
y = np.array([l],dtype=np.int)

z = np.hstack((x, y))

print(z.shape)
print(type(z))

(11,)
<class 'numpy.ndarray'>


In [253]:
x='{}'.format(int(0))
int(x)

0

In [267]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-pytorch-2019-01-26-15-39-02-195
INFO:sagemaker:Creating endpoint with name sagemaker-pytorch-2019-01-26-15-39-02-195


-----------------------------------------------------------!

### Testing the model

Now that we have deployed our model with the custom inference code, we should test to see if everything is working. Here we test our model by loading the first `250` positive and negative reviews and send them to the endpoint, then collect the results. The reason for only sending some of the data is that the amount of time it takes for our model to process the input and then perform inference is quite long and so testing the entire data set would be prohibitive.

In [257]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
               # print(predictor.predict(review_input))
                results.append(int(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [258]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [259]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.816

As an additional test, we can try sending the `test_review` that we looked at earlier.

In [260]:
predictor.predict(test_review)

b'1'

Now that we know our endpoint is working as expected, we can set up the web page that will interact with it. If you don't have time to finish the project now, make sure to skip down to the end of this notebook and shut down your endpoint. You can deploy it again when you come back.

## Step 7 (again): Use the model for the web app

> **TODO:** This entire section and the next contain tasks for you to complete, mostly using the AWS console.

So far we have been accessing our model endpoint by constructing a predictor object which uses the endpoint and then just using the predictor object to perform inference. What if we wanted to create a web app which accessed our model? The way things are set up currently makes that not possible since in order to access a SageMaker endpoint the app would first have to authenticate with AWS using an IAM role which included access to SageMaker endpoints. However, there is an easier way! We just need to use some additional AWS services.

<img src="Web App Diagram.svg">

The diagram above gives an overview of how the various services will work together. On the far right is the model which we trained above and which is deployed using SageMaker. On the far left is our web app that collects a user's movie review, sends it off and expects a positive or negative sentiment in return.

In the middle is where some of the magic happens. We will construct a Lambda function, which you can think of as a straightforward Python function that can be executed whenever a specified event occurs. We will give this function permission to send and recieve data from a SageMaker endpoint.

Lastly, the method we will use to execute the Lambda function is a new endpoint that we will create using API Gateway. This endpoint will be a url that listens for data to be sent to it. Once it gets some data it will pass that data on to the Lambda function and then return whatever the Lambda function returns. Essentially it will act as an interface that lets our web app communicate with the Lambda function.

### Setting up a Lambda function

The first thing we are going to do is set up a Lambda function. This Lambda function will be executed whenever our public API has data sent to it. When it is executed it will receive the data, perform any sort of processing that is required, send the data (the review) to the SageMaker endpoint we've created and then return the result.

#### Part A: Create an IAM Role for the Lambda function

Since we want the Lambda function to call a SageMaker endpoint, we need to make sure that it has permission to do so. To do this, we will construct a role that we can later give the Lambda function.

Using the AWS Console, navigate to the **IAM** page and click on **Roles**. Then, click on **Create role**. Make sure that the **AWS service** is the type of trusted entity selected and choose **Lambda** as the service that will use this role, then click **Next: Permissions**.

In the search box type `sagemaker` and select the check box next to the **AmazonSageMakerFullAccess** policy. Then, click on **Next: Review**.

Lastly, give this role a name. Make sure you use a name that you will remember later on, for example `LambdaSageMakerRole`. Then, click on **Create role**.

#### Part B: Create a Lambda function

Now it is time to actually create the Lambda function.

Using the AWS Console, navigate to the AWS Lambda page and click on **Create a function**. When you get to the next page, make sure that **Author from scratch** is selected. Now, name your Lambda function, using a name that you will remember later on, for example `sentiment_analysis_func`. Make sure that the **Python 3.6** runtime is selected and then choose the role that you created in the previous part. Then, click on **Create Function**.

On the next page you will see some information about the Lambda function you've just created. If you scroll down you should see an editor in which you can write the code that will be executed when your Lambda function is triggered. In our example, we will use the code below. 

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```

Once you have copy and pasted the code above into the Lambda code editor, replace the `**ENDPOINT NAME HERE**` portion with the name of the endpoint that we deployed earlier. You can determine the name of the endpoint using the code cell below.

In [268]:
predictor.endpoint

'sagemaker-pytorch-2019-01-26-15-39-02-195'

Once you have added the endpoint name to the Lambda function, click on **Save**. Your Lambda function is now up and running. Next we need to create a way for our web app to execute the Lambda function.

### Setting up API Gateway

Now that our Lambda function is set up, it is time to create a new API using API Gateway that will trigger the Lambda function we have just created.

Using AWS Console, navigate to **Amazon API Gateway** and then click on **Get started**.

On the next page, make sure that **New API** is selected and give the new api a name, for example, `sentiment_analysis_api`. Then, click on **Create API**.

Now we have created an API, however it doesn't currently do anything. What we want it to do is to trigger the Lambda function that we created earlier.

Select the **Actions** dropdown menu and click **Create Method**. A new blank method will be created, select its dropdown menu and select **POST**, then click on the check mark beside it.

For the integration point, make sure that **Lambda Function** is selected and click on the **Use Lambda Proxy integration**. This option makes sure that the data that is sent to the API is then sent directly to the Lambda function with no processing. It also means that the return value must be a proper response object as it will also not be processed by API Gateway.

Type the name of the Lambda function you created earlier into the **Lambda Function** text entry box and then click on **Save**. Click on **OK** in the pop-up box that then appears, giving permission to API Gateway to invoke the Lambda function you created.

The last step in creating the API Gateway is to select the **Actions** dropdown and click on **Deploy API**. You will need to create a new Deployment stage and name it anything you like, for example `prod`.

You have now successfully set up a public API to access your SageMaker model. Make sure to copy or write down the URL provided to invoke your newly created public API as this will be needed in the next step. This URL can be found at the top of the page, highlighted in blue next to the text **Invoke URL**.

## Step 4: Deploying our web app

Now that we have a publicly available API, we can start using it in a web app. For our purposes, we have provided a simple static html file which can make use of the public api you created earlier.

In the `website` folder there should be a file called `index.html`. Download the file to your computer and open that file up in a text editor of your choice. There should be a line which contains **\*\*REPLACE WITH PUBLIC API URL\*\***. Replace this string with the url that you wrote down in the last step and then save the file.

Now, if you open `index.html` on your local computer, your browser will behave as a local web server and you can use the provided site to interact with your SageMaker model.

If you'd like to go further, you can host this html file anywhere you'd like, for example using github or hosting a static site on Amazon's S3. Once you have done this you can share the link with anyone you'd like and have them play with it too!

> **Important Note** In order for the web app to communicate with the SageMaker endpoint, the endpoint has to actually be deployed and running. This means that you are paying for it. Make sure that the endpoint is running when you want to use the web app but that you shut it down when you don't need it, otherwise you will end up with a surprisingly large AWS bill.

**TODO:** Make sure that you include the edited `index.html` file in your project submission.

Now that your web app is working, trying playing around with it and see how well it works.

**Question**: Give an example of a review that you entered into your web app. What was the predicted sentiment of your example review?

**Answer:**
"Fifty Shades of Grey" is based on a badly written book, so why someone thought they could make a decent film out of it is beyond me. Actually I don't think anyone believed they could make a good film, the producers just wanted the money that would come in from all people who for reasons known only to themselves read the book.

I had a free ticket and I had to go in order to do a story on it. Bottom line, I'm not doing a story on it.

Okay, here's the problem, and this is what breaks my heart. We no longer know what good or great is. I am no follower of Ayn Rand, but in The Fountainhead, she predicted the rise of mediocrity, mediocrity being considered great and the norm we shoot for. It only stands to reason that at this point, some things can't even rise to that sad level. This isn't mediocre, it's pathetic. I can't believe there are people complimenting the actors. But then I can, because they have come to accept mediocrity. Something I fight against with all my heart.

There is one other problem. The book sucked; in order to like it, one had to use his or her imagination. The success of this book lay in the fantasy aspect. Sometimes imagination is more powerful than just about anything and can give you an erotic experience. This film doesn't fulfill the most inane imagination, let alone someone who really has a wild and creative one. The only fantasy it can give you is how fast you can get out of the theater.

There were people around me laughing. The ones who weren't left, and from what I found out in the lobby, actually demanded and got their money back. When was the last time you saw people leaving a movie in the middle? I'm not sure I've ever seen it, and I'll guess I've been going to movies longer than a lot of you.

I lament the loss of artistic soul, of creativity, of going for the best. I miss the days when Billy Wilder was a writer and a filmmaker. What are we left with but a horrible book written on a first grade level and the resulting bad movie.


1 star review => predicted "Your review was NEGATIVE!"


### Delete the endpoint

Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill.

In [ ]:
predictor.delete_endpoint()